# Interview Question Generation
`Author: Abdlazeez Jimoh`

In [1]:
import json
from pprint import pprint
from types import ModuleType
from typing import Any, Literal

import cohere  # type: ignore
import google.generativeai as GooglePalm  # type: ignore
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletion
from pydantic import BaseModel

/home/abdulazeezoj/Devspace/Work/omdena/local-chapter/india-chapter/chatbot-interview-prep/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
COHERE_API_KEY = "YOUR_COHERE_API_KEY"
OPENAI_API_KEY = "YOUR_OPENAI_API_KEY"
PALM_API_KEY = "YOUR_PALM_API_KEY"

In [3]:
class Question(BaseModel):
    question: str
    type: Literal["personal", "role-specific", "behavioral", "situational"]

## OpenAI Interview Question Generation

In [4]:
class OpenAIQuestionGeneratorAgent:
    def __init__(self) -> None:
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.system_prompt = """You are a non-technical interviewer that interviews \
across the following categories:
- personal
- role-specific
- behavioural
- situational

You will be provided with a candidate's description.

Generate {n_questions} questions, ensuring that there is a question for each category \
and the questions should be based on the candidate's description.

* You answer strictly as a list of JSON objects. Don't include any other verbose texts, \
and don't include the markdown syntax anywhere.

JSON format:
[
    {{"question": "<personal_question>", "type": "personal"}},
    {{"question": "<role_specific_question>", "type": "role-specific"}},
    {{"question": "<behavioural_question>", "type": "behavioural"}},
    {{"question": "<situational_question>", "type": "situational"}},
    ...more questions to make up {n_questions} questions
]"""

        self.user_prompt = "Candidate Description:\n{description}"

    def __call__(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        # Generate questions
        questions: list[Question] | None = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        # Generate questions
        questions: list[Question] | None = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        try:
            # Ensure that there are at least 4 questions
            if n_questions < 4:
                n_questions = 4

            output: ChatCompletion = self.client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {
                        "role": "system",
                        "content": self.system_prompt.format(n_questions=n_questions),
                    },
                    {
                        "role": "user",
                        "content": self.user_prompt.format(description=description),
                    },
                ],
                temperature=0.5,
                max_tokens=1024,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )

            if output.choices[0].finish_reason != "stop":
                return None

            if not output.choices[0].message.content:
                return None

            questions: list[Question] = json.loads(output.choices[0].message.content)

            return questions
        except Exception:
            return None

In [5]:
question_generator = OpenAIQuestionGeneratorAgent()
questions: list[Question] | None = question_generator.run(
    "a software engineer at a startup in San Francisco. I have 5 years of "
    "experience and I'm looking for a new job.",
    5,
)

pprint(questions)

[{'question': 'What motivated you to pursue a career in software engineering?',
  'type': 'personal'},
 {'question': 'Can you describe a challenging technical problem you solved at '
              'your previous startup?',
  'type': 'role-specific'},
 {'question': 'How do you handle tight deadlines and multiple projects '
              'simultaneously?',
  'type': 'behavioural'},
 {'question': 'If you were the only software engineer on a project and faced a '
              'roadblock, how would you approach solving the issue?',
  'type': 'situational'},
 {'question': 'How do you stay updated with the latest technologies and '
              'industry trends in software engineering?',
  'type': 'role-specific'}]


In [6]:
questions = question_generator.run(
    "a data scientist from India. I have 3 years of experience. I've worked on a variety of projects, including a recommendation engine for a large e-commerce company. I've led a team of 5 data scientists and engineers.",
    8,
)

pprint(questions)

[{'question': 'What inspired you to pursue a career in data science?',
  'type': 'personal'},
 {'question': 'Can you describe a specific project where you implemented a '
              'recommendation engine?',
  'type': 'role-specific'},
 {'question': 'How do you typically approach problem-solving in your data '
              'science work?',
  'type': 'behavioural'},
 {'question': 'Tell me about a time when you had to lead a team of data '
              'scientists and engineers to accomplish a project goal.',
  'type': 'situational'},
 {'question': 'What are the key skills you believe are essential for a '
              'successful data scientist?',
  'type': 'personal'},
 {'question': 'Can you provide an example of a challenging problem you '
              'encountered in a data science project and how you overcame it?',
  'type': 'behavioural'},
 {'question': 'How do you prioritize tasks and manage deadlines when leading a '
              'team on multiple projects?',
  'type': 's

## Google Palm Interview Question Generation

In [7]:
class PalmQuestionGeneratorAgent:
    def __init__(self) -> None:
        self.client: ModuleType = GooglePalm
        self.client.configure(api_key=PALM_API_KEY)  # type: ignore
        self.system_prompt = """You are a non-technical interviewer that interviews \
across the following categories:
- personal
- role-specific
- behavioural
- situational

You will be provided with a candidate's description.

Generate {n_questions} questions, ensuring that there is a question for each category \
and the questions should be based on the candidate's description.

* You answer strictly as a list of JSON objects. Don't include any other verbose texts, \
and don't include the markdown syntax anywhere.

JSON format:
[
    {{"question": "<personal_question>", "type": "personal"}},
    {{"question": "<role_specific_question>", "type": "role-specific"}},
    {{"question": "<behavioural_question>", "type": "behavioural"}},
    {{"question": "<situational_question>", "type": "situational"}},
    ...more questions to make up {n_questions} questions
]


===
Candidate Description:
{description}"""

    def __call__(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        # Generate questions
        questions: list[Question] | None = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        # Generate questions
        questions: list[Question] | None = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        try:
            # Ensure that there are at least 4 questions
            if n_questions < 4:
                n_questions = 4

            output: Any = self.client.generate_text(
                model="models/text-bison-001",
                prompt=self.system_prompt.format(
                    n_questions=n_questions, description=description
                ),
                temperature=1,
                max_output_tokens=1024,
            )

            if not output.result:
                return None

            questions: list[Question] = json.loads(output.result)

            return questions
        except Exception:
            return None

In [8]:
question_generator = PalmQuestionGeneratorAgent()
questions: list[Question] | None = question_generator.run(
    "a software engineer at a startup in San Francisco. I have 5 years of "
    "experience and I'm looking for a new job.",
    5,
)

pprint(questions)

[{'question': 'What are your career goals?', 'type': 'personal'},
 {'question': 'What are your salary expectations?', 'type': 'role-specific'},
 {'question': 'Tell me about a time when you had to work on a team project and '
              'you disagreed with a teammate about how to proceed. How did you '
              'resolve the disagreement?',
  'type': 'behavioural'},
 {'question': "What would you do if you were given a project that you didn't "
              'have the skills to complete?',
  'type': 'situational'},
 {'question': 'Why did you leave your last job?', 'type': 'personal'}]


In [9]:
questions: list[Question] | None = question_generator.run(
    (
        "a data scientist from India. I have 3 years of experience. I've worked on "
        "a variety of projects, including a recommendation engine for a large "
        "e-commerce company. I've led a team of 5 data scientists and engineers."
    ),
    8,
)

pprint(questions)

[{'question': 'What is your greatest weakness?', 'type': 'personal'},
 {'question': 'What is your greatest strength?', 'type': 'personal'},
 {'question': 'What are your salary expectations?', 'type': 'personal'},
 {'question': 'What are your long-term career goals?', 'type': 'personal'},
 {'question': 'Why are you interested in this position?',
  'type': 'role-specific'},
 {'question': 'What are your experiences with machine learning?',
  'type': 'role-specific'},
 {'question': 'What are your experiences with deep learning?',
  'type': 'role-specific'},
 {'question': 'How would you handle a situation where your team is not meeting '
              'deadlines?',
  'type': 'situational'},
 {'question': 'How would you handle a situation where your boss is asking you '
              'to do something that you believe is unethical?',
  'type': 'situational'},
 {'question': 'What is your greatest accomplishment?', 'type': 'behavioural'},
 {'question': 'What is your biggest failure?', 'type': '

## Cohere Interview Question Generation

In [10]:
class CohereQuestionGeneratorAgent:
    def __init__(self) -> None:
        self.client: Any = cohere.Client(api_key=COHERE_API_KEY)  # type: ignore
        self.system_prompt = """You are a non-technical interviewer that interviews \
across the following categories:
- personal
- role-specific
- behavioural
- situational

You will be provided with a candidate's description.

Generate {n_questions} questions, ensuring that there is a question for each category \
and the questions should be based on the candidate's description.

* You answer strictly as a list of JSON objects. Don't include any other verbose texts, \
and don't include the markdown syntax anywhere.

JSON format:
[
    {{"question": "<personal_question>", "type": "personal"}},
    {{"question": "<role_specific_question>", "type": "role-specific"}},
    {{"question": "<behavioural_question>", "type": "behavioural"}},
    {{"question": "<situational_question>", "type": "situational"}},
    ...more questions to make up {n_questions} questions
]


===
Candidate Description:
{description}"""

    def __call__(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        # Generate questions
        questions: list[Question] | None = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 4) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        # Generate questions
        questions: list[Question] | None = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> list[Question] | None:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            list[Question] | None: A list of generated interview questions or \
                None if an error occurs.
        """

        try:
            # Ensure that there are at least 4 questions
            if n_questions < 4:
                n_questions = 4

            output: Any = self.client.generate(
                model="command",
                prompt=self.system_prompt.format(
                    n_questions=n_questions, description=description
                ),
                temperature=1,
                max_tokens=1024,
            )

            if not output.generations:
                return None

            if not output.generations[0].text:
                return None

            questions: list[Question] = json.loads(output.generations[0].text)

            return questions
        except Exception:
            return None

In [13]:
question_generator = CohereQuestionGeneratorAgent()
questions: list[Question] | None = question_generator.run(
    (
        "a software engineer at a startup in San Francisco. I have 5 years of "
        "experience and I'm looking for a new job."
    ),
    5,
)

pprint(questions)

[{'question': 'What motivated you to leave your previous role and seek out a '
              'new opportunity?',
  'type': 'personal'},
 {'question': 'Can you describe a time when you had to adapt to a major change '
              'in your engineering field? What was the cause of the change and '
              'what was your role in implementing it?',
  'type': 'role-specific'},
 {'question': 'Tell me about a time you had a conflict with a colleague. How '
              'did you resolve the situation?',
  'type': 'behavioural'},
 {'question': 'What do you find most challenging about working in the San '
              'Francisco tech scene?',
  'type': 'situational'},
 {'question': 'Describe your experience with version control systems (e.g. '
              'Git). How do you manage your repositories and what features do '
              'you frequently use?',
  'type': 'role-specific'}]


In [12]:
questions: list[Question] | None = question_generator.run(
    "a data scientist from India. I have 3 years of experience. I've worked "
    "on a variety of projects, including a recommendation engine for a "
    "large e-commerce company. I've led a team of 5 data scientists and "
    "engineers.",
    8,
)

pprint(questions)

[{'question': 'What motivated you to pursue a career in data science?',
  'type': 'personal'},
 {'question': 'Can you describe a time when you used Python to solve a complex '
              'data problem? What libraries or frameworks did you use, and '
              'what was your approach to the problem?',
  'type': 'role-specific'},
 {'question': 'How do you keep up to date with industry trends and '
              'developments in data science?',
  'type': 'personal'},
 {'question': 'Can you explain your experience with recommendation engines? '
              'What challenges did you face and how did you overcome them?',
  'type': 'role-specific'},
 {'question': 'What project have you worked on that you are most proud of and '
              'why?',
  'type': 'personal'},
 {'question': 'Tell me about a time when you used data visualization to solve '
              'a problem. What kind of data did you work with, and what '
              'visualization techniques did you use to communi